In [1]:
!pip install pandas-highcharts

  Created wheel for pandas-highcharts: filename=pandas_highcharts-0.5.2-py3-none-any.whl size=7809 sha256=9c487735539a374c96c0037824473b2756218727eebda12863da7102fbef5c31
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\b1\83\f5\038ee7d34b53811b50f587b60a4cd07954917d9b648c07d7c6
Successfully built pandas-highcharts


In [3]:
import os
import datetime

import IPython
import IPython.display
import requests
import json
import time
import math
import re
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from  matplotlib.ticker import PercentFormatter
import seaborn as sns
 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
 
from sklearn import set_config
# set_config(display='diagram') # Để trực quan hóa pipeline
 
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.utils import resample

from pandas_highcharts.display import display_charts
from pandas_highcharts.display import load_highcharts
from pandas_highcharts.core import serialize
from IPython.core.display import display, HTML
from pandas_highcharts.core import json_encode

In [4]:
#Set Option
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_theme()

In [8]:
df = pd.read_excel('data/data.xls')

In [10]:
df.head(3)

,TÊN THỨC ĂN,Calories (kcal),Protein (g),Fat (g),Carbonhydrates (g),Chất xơ (g),Cholesterol (mg),Canxi (mg),Photpho (mg),Sắt (mg),Natri (mg),Kali (mg),Beta Caroten (mcg),Vitamin A (mcg),Vitamin B1 (mg),Vitamin C (mg),Loại
0,Gạo nếp cái,346,"8,6","1,5","74,9","0,6",0,32,98,"1,2",3,282,0,0,"0,14","0,0",Ngũ cốc và sản phẩm chế biến từ chúng
1,Gạo tẻ,344,"7,9","1,0","76,2","0,4",0,30,"104,0","1,3",5,241,0,0,"0,10","0,0",Ngũ cốc và sản phẩm chế biến từ chúng
2,Bắp tươi,196,"4,1","2,3","39,6","1,2",0,20,"187,0","1,5",0,0,170,0,"0,21","0,0",Ngũ cốc và sản phẩm chế biến từ chúng
